In [1]:
from pathlib import Path

print("CWD =", Path.cwd())

search_roots = [Path.cwd(), Path("work"), Path("work/data"), Path("data")]
patterns = [
    "*uk-englandwales-ndr-2010*listentries*baseline*csv*",
    "*uk-englandwales-ndr-2017*listentries*baseline*csv*",
    "*uk-englandwales-ndr-2023*listentries*baseline*csv*"
]

# 递归搜索
hits = []
for root in search_roots:
    if not root.exists():
        continue
    for pat in patterns:
        hits.extend(root.rglob(pat))

# 去重 + 只要存在的文件
seen, files = set(), []
for p in hits:
    try:
        rp = p.resolve()
    except Exception:
        rp = p
    s = str(rp)
    if s not in seen and p.exists():
        seen.add(s)
        files.append(p)

if files:
    print("\nFound files:")
    for i, p in enumerate(files, 1):
        try:
            size_mb = p.stat().st_size / (1024*1024)
        except Exception as e:
            size_mb = float('nan')
        print(f"{i:>2}. {p.resolve()}  ({size_mb:.1f} MB)")
else:
    print("\nNo matches found. Quick peek under 'work/data/raw' (1 level):")
    base = Path("work/data/raw")
    if base.exists():
        for x in sorted(base.iterdir()):
            print(" -", x)
    else:
        print(" - 'work/data/raw' not found.")

CWD = /home/jovyan/work

Found files:
 1. /home/jovyan/work/data/raw/voa_list_entries_2010_epoch0053/uk-englandwales-ndr-2010-listentries-compiled-epoch-0053-baseline-csv.csv  (650.9 MB)
 2. /home/jovyan/work/data/raw/voa_list_entries_2010_epoch0053/uk-englandwales-ndr-2010-listentries-compiled-epoch-0053-baseline-historic-csv.csv  (195.6 MB)
 3. /home/jovyan/work/data/raw/voa_list_entries_2017_epoch0052/uk-englandwales-ndr-2017-listentries-compiled-epoch-0052-baseline-csv.csv  (638.8 MB)
 4. /home/jovyan/work/data/raw/voa_list_entries_2017_epoch0052/uk-englandwales-ndr-2017-listentries-compiled-epoch-0052-baseline-csv_london.csv  (0.0 MB)
 5. /home/jovyan/work/data/raw/voa_list_entries_2017_epoch0052/uk-englandwales-ndr-2017-listentries-compiled-epoch-0052-baseline-csv_london_by_bacode.csv  (91.3 MB)
 6. /home/jovyan/work/data/raw/voa_list_entries_2017_epoch0052/uk-englandwales-ndr-2017-listentries-compiled-epoch-0052-baseline-historic-csv.csv  (125.4 MB)
 7. /home/jovyan/work/data/ra

In [7]:
import pandas as pd, csv, re, time
from pathlib import Path

FILES = [
    Path("/home/jovyan/work/data/raw/voa_list_entries_2010_epoch0053/uk-englandwales-ndr-2010-listentries-compiled-epoch-0053-baseline-csv.csv"),
    Path("/home/jovyan/work/data/raw/voa_list_entries_2017_epoch0052/uk-englandwales-ndr-2017-listentries-compiled-epoch-0052-baseline-csv.csv"),
    Path("/home/jovyan/work/data/raw/voa_list_entries_2023_epoch0016/uk-englandwales-ndr-2023-listentries-compiled-epoch-0016-baseline-csv.csv"),
]

# 伦敦 Billing Authority Codes（官方附录1）
LONDON_BA = {
    "5030":"City of London","5060":"Barking & Dagenham","5090":"Barnet","5120":"Bexley",
    "5150":"Brent","5180":"Bromley","5210":"Camden","5240":"Croydon","5270":"Ealing",
    "5300":"Enfield","5330":"Greenwich","5360":"Hackney","5390":"Hammersmith & Fulham",
    "5420":"Haringey","5450":"Harrow","5480":"Havering","5510":"Hillingdon",
    "5540":"Hounslow","5570":"Islington","5600":"Kensington & Chelsea",
    "5630":"Kingston upon Thames","5660":"Lambeth","5690":"Lewisham","5720":"Merton",
    "5750":"Newham","5780":"Redbridge","5810":"Richmond upon Thames","5840":"Southwark",
    "5870":"Sutton","5900":"Tower Hamlets","5930":"Waltham Forest","5960":"Wandsworth",
    "5990":"Westminster City"
}

ENCODINGS = ["ascii","cp1252","latin1","utf-8","utf-8-sig"]

def guess_cols(fp):
    # VOA 无表头：按规范顺序，V02 通常是 Billing Authority Code
    # 读取首行确定列数
    with open(fp, "rb") as f: first = f.readline()
    first = first.decode("latin1").strip("\r\n")
    n = len(first.split("*"))
    names = [f"V{i:02d}" for i in range(1, n+1)]
    return names, "V02"

def filter_by_bacode(fp: Path, chunksize=150_000):
    names, ba_col = guess_cols(fp)
    out_csv = fp.with_suffix("").as_posix() + "_london_by_bacode.csv"
    first_write, enc_used = True, None
    total = kept = 0
    counts = {}

    for enc in ENCODINGS:
        try:
            for ch in pd.read_csv(fp, sep="*", names=names, header=None, dtype=str,
                                   engine="python", encoding=enc, quoting=csv.QUOTE_NONE,
                                   chunksize=chunksize, on_bad_lines="skip"):
                enc_used = enc
                total += len(ch)
                mask = ch[ba_col].isin(LONDON_BA.keys())
                sub = ch.loc[mask].copy()
                kept += len(sub)
                # 计数各区
                for code, n in sub[ba_col].value_counts().items():
                    counts[code] = counts.get(code, 0) + int(n)
                sub.to_csv(out_csv, index=False, mode="w" if first_write else "a",
                           header=first_write, encoding="utf-8-sig")
                first_write = False
            break
        except UnicodeDecodeError:
            continue

    print(f"\n[{fp.name}] encoding={enc_used}  total={total:,}  London kept={kept:,}  → {out_csv}")
    # 打印各区 Top 10 和合计
    s = pd.Series(counts).sort_values(ascending=False)
    if not s.empty:
        pretty = s.rename(index=LONDON_BA).head(10)
        print("\nTop boroughs by rows:\n", pretty.to_string())
        print("\nUnique boroughs captured:", len(s), "/ 33")

for f in FILES:
    filter_by_bacode(f)


[uk-englandwales-ndr-2010-listentries-compiled-epoch-0053-baseline-csv.csv] encoding=cp1252  total=2,764,077  London kept=418,268  → /home/jovyan/work/data/raw/voa_list_entries_2010_epoch0053/uk-englandwales-ndr-2010-listentries-compiled-epoch-0053-baseline-csv_london_by_bacode.csv

Top boroughs by rows:
 Westminster City        50751
City of London          29541
Camden                  25006
Tower Hamlets           24307
Islington               16565
Southwark               16067
Hackney                 14515
Ealing                  14331
Hillingdon              13176
Hammersmith & Fulham    12249

Unique boroughs captured: 33 / 33

[uk-englandwales-ndr-2017-listentries-compiled-epoch-0052-baseline-csv.csv] encoding=cp1252  total=2,719,851  London kept=413,120  → /home/jovyan/work/data/raw/voa_list_entries_2017_epoch0052/uk-englandwales-ndr-2017-listentries-compiled-epoch-0052-baseline-csv_london_by_bacode.csv

Top boroughs by rows:
 Westminster City        50304
City of London     

In [8]:
# -*- coding: utf-8 -*-
import pandas as pd, re
from pathlib import Path

# —— 输入（已是 *_london_by_bacode.csv）——
F2010 = Path("/home/jovyan/work/data/raw/voa_list_entries_2010_epoch0053/uk-englandwales-ndr-2010-listentries-compiled-epoch-0053-baseline-csv_london_by_bacode.csv")
F2017 = Path("/home/jovyan/work/data/raw/voa_list_entries_2017_epoch0052/uk-englandwales-ndr-2017-listentries-compiled-epoch-0052-baseline-csv_london_by_bacode.csv")
F2023 = Path("/home/jovyan/work/data/raw/voa_list_entries_2023_epoch0016/uk-englandwales-ndr-2023-listentries-compiled-epoch-0016-baseline-csv_london_by_bacode.csv")

OUTDIR = Path("/home/jovyan/work/data/processed/voa")
OUTDIR.mkdir(parents=True, exist_ok=True)

# —— 列位（中间件列名）——
KEY_COL  = "V07"   # UARN
BA_COL   = "V02"   # Billing Authority Code
RV_COL   = "V18"   # Rateable Value
SCAT_COL = "V22"   # SCAT（取前三位）
PC_COL   = "V15"   # Postcode → 重命名为 postcode

# —— 零售 CORE 口径（窄）——
SCAT_KEEP = {
    '008','011','021','024','086','097','098','106','139','152','154','155',
    '165','210','235','243','249','251','417','425','429','442'
}

def make_retail_core(in_path: Path, year: int, chunksize=200_000) -> Path:
    """不识别变更：零售筛选 + RV/UARN 清洗 + 同一UARN取最大RV。"""
    out = OUTDIR / f"voa_{year}_london_retail_core.csv"
    first, kept, scanned = True, 0, 0
    usecols = [KEY_COL, BA_COL, RV_COL, SCAT_COL, PC_COL]

    for ch in pd.read_csv(in_path, dtype=str, encoding="utf-8-sig",
                          usecols=usecols, chunksize=chunksize):
        # SCAT 三位筛选（零售）
        scat3 = ch[SCAT_COL].astype(str).str.extract(r'^(\d{3})', expand=False)
        scanned += len(scat3)
        ch = ch.loc[scat3.isin(SCAT_KEEP)].copy()
        if ch.empty: 
            continue

        # 清洗
        ch["rv"] = pd.to_numeric(
            ch[RV_COL].astype(str).str.replace(r'\D','',regex=True), errors="coerce"
        )
        ch["uarn"] = ch[KEY_COL].astype(str).str.replace(r'\D','',regex=True)
        ch = ch[ch["uarn"].str.len().between(7,16)]
        ch = ch[ch["rv"].fillna(0) > 0]
        ch = ch.rename(columns={PC_COL: "postcode"})

        # 同一 UARN 取最大 RV
        ch = ch.sort_values("rv").drop_duplicates("uarn", keep="last")

        kept += len(ch)
        ch[["uarn", BA_COL, SCAT_COL, "rv", "postcode"]].to_csv(
            out, index=False, mode="w" if first else "a",
            header=first, encoding="utf-8-sig"
        )
        first = False

    print(f"[{year}] kept {kept:,} / scanned {scanned:,} → {out}")
    return out

# —— 执行 —— 
if __name__ == "__main__":
    p10 = make_retail_core(F2010, 2010)
    p17 = make_retail_core(F2017, 2017)
    p23 = make_retail_core(F2023, 2023)
    print("Done:", p10, "|", p17, "|", p23)


[2010] kept 90,306 / scanned 418,268 → /home/jovyan/work/data/processed/voa/voa_2010_london_retail_core.csv
[2017] kept 90,398 / scanned 413,120 → /home/jovyan/work/data/processed/voa/voa_2017_london_retail_core.csv
[2023] kept 90,107 / scanned 343,487 → /home/jovyan/work/data/processed/voa/voa_2023_london_retail_core.csv
Done: /home/jovyan/work/data/processed/voa/voa_2010_london_retail_core.csv | /home/jovyan/work/data/processed/voa/voa_2017_london_retail_core.csv | /home/jovyan/work/data/processed/voa/voa_2023_london_retail_core.csv


In [55]:
import pandas as pd
from pathlib import Path

DIR = Path("/home/jovyan/work/data/processed/voa")
P10 = DIR / "voa_2010_london_retail_core.csv"
P17 = DIR / "voa_2017_london_retail_core.csv"
P23 = DIR / "voa_2023_london_retail_core.csv"

def norm_uarn(s: pd.Series) -> pd.Series:
    return s.astype(str).str.replace(r"\D", "", regex=True)

# 读取并对齐 UARN
d10 = pd.read_csv(P10, usecols=["uarn"], dtype={"uarn": str})
d17 = pd.read_csv(P17, usecols=["uarn"], dtype={"uarn": str})
d23 = pd.read_csv(P23, usecols=["uarn"], dtype={"uarn": str})
d10["uarn"] = norm_uarn(d10["uarn"])
d17["uarn"] = norm_uarn(d17["uarn"])
d23["uarn"] = norm_uarn(d23["uarn"])

# 集合
s10, s17, s23 = set(d10.uarn), set(d17.uarn), set(d23.uarn)
union_all = s10 | s17 | s23
den = len(union_all)

if den == 0:
    for _ in range(7): print("0.000000")
else:
    only_10    = s10 - s17 - s23
    only_17    = s17 - s10 - s23
    only_23    = s23 - s10 - s17
    only_10_17 = (s10 & s17) - s23
    only_17_23 = (s17 & s23) - s10
    only_10_23 = (s10 & s23) - s17
    all_three  =  s10 & s17 & s23

    print("only_10",f"{len(only_10)/den:.6f}")
    print("only_17",f"{len(only_17)/den:.6f}")
    print("only_23",f"{len(only_23)/den:.6f}")
    print("only_10_17",f"{len(only_10_17)/den:.6f}")
    print("only_17_23",f"{len(only_17_23)/den:.6f}")
    print("only_10_23",f"{len(only_10_23)/den:.6f}")
    print("all_three",f"{len(all_three)/den:.6f}")
   


only_10 0.099398
only_17 0.006294
only_23 0.021070
only_10_17 0.017773
only_17_23 0.094164
only_10_23 0.000165
all_three 0.761136


In [24]:
!pip -q install duckdb
import duckdb, pathlib
BASE = pathlib.Path("/home/jovyan/work/data/processed/voa")
con = duckdb.connect()

In [30]:
import duckdb, pathlib

BASE = pathlib.Path("/home/jovyan/work/data/processed/voa")
f10 = str(BASE/"voa_2010_london_retail_core.csv")
f17 = str(BASE/"voa_2017_london_retail_core.csv")
f23 = str(BASE/"voa_2023_london_retail_core.csv")

SCAT_GROUPS = {
    "SMALL_SHOP":   {"249","106"},
    "SUPERMARKET":  {"139","152"},
    "LARGE_NONFOOD":{"154","155","086","251"},
    "RETAIL_WARE":  {"235"},
    "KIOSK":        {"243"},
    "PHARMACY_PO":  {"210","425","429"},
    "MARKET":       {"165"},
    "FIN_BET_BEAUTY":{"021","024","417"},
    "TAKEAWAY":     {"442"},
    "LEISURE_AIR":  {"008","011"},
}
case_lines = [f"WHEN scat3='{c}' THEN '{g}'" for g,cs in SCAT_GROUPS.items() for c in cs]
SCAT_CASE = " ".join(case_lines)

con = duckdb.connect()
# 可省略：让 DuckDB 用默认线程数
# con.execute(f"PRAGMA threads={int(os.cpu_count() or 4)};")

# ① 直接把 WITH 包在 COPY(..) 里 —— 不会再报分号错误
con.execute(f"""
COPY (
WITH src AS (
  SELECT 2008 AS avd, * FROM read_csv_auto('{f10}', HEADER=TRUE, ALL_VARCHAR=TRUE, SAMPLE_SIZE=-1)
  UNION ALL
  SELECT 2015 AS avd, * FROM read_csv_auto('{f17}', HEADER=TRUE, ALL_VARCHAR=TRUE, SAMPLE_SIZE=-1)
  UNION ALL
  SELECT 2021 AS avd, * FROM read_csv_auto('{f23}', HEADER=TRUE, ALL_VARCHAR=TRUE, SAMPLE_SIZE=-1)
),
norm AS (
  SELECT
    avd,
    NULLIF(regexp_replace(uarn, '[^0-9]', '', 'g'), '') AS uarn,
    "V02" AS bacode,
    "V22" AS scat,
    TRY_CAST(rv AS DOUBLE) AS rv,
    UPPER(regexp_replace(postcode, '[^A-Z0-9]', '', 'g')) AS pc0
  FROM src
),
norm2 AS (
  SELECT
    avd, uarn, bacode, scat,
    regexp_extract(scat, '(\\d{{3}})', 1) AS scat3,
    rv,
    CASE
      WHEN pc0='GIR0AA' THEN 'GIR 0AA'
      WHEN length(pc0) BETWEEN 5 AND 7
           AND regexp_full_match(right(pc0,3), '[0-9][A-Z]{{2}}')
        THEN left(pc0, length(pc0)-3) || ' ' || right(pc0,3)
      ELSE NULLIF(pc0,'')
    END AS postcode
  FROM norm
),
panel AS (
  SELECT
    avd,
    uarn,
    postcode,
    bacode AS borough_i,
    scat,
    scat3,
    CASE {SCAT_CASE} ELSE scat3 END AS scat_grp,
    rv,
    CASE WHEN rv > 0 THEN ln(rv) END AS ln_rv
  FROM norm2
),
long AS (
  SELECT
    p.*,
    COUNT(DISTINCT avd) OVER (PARTITION BY uarn)                                  AS n_t,
    (COUNT(DISTINCT avd) OVER (PARTITION BY uarn)=3)::INT                          AS balanced_3,
    (SUM((avd=2008)::INT) OVER (PARTITION BY uarn)>=1 AND
     SUM((avd=2015)::INT) OVER (PARTITION BY uarn)>=1)::INT                        AS pair_0815,
    (SUM((avd=2015)::INT) OVER (PARTITION BY uarn)>=1 AND
     SUM((avd=2021)::INT) OVER (PARTITION BY uarn)>=1)::INT                        AS pair_1521,
    (SUM((avd=2008)::INT) OVER (PARTITION BY uarn)>=1 AND
     SUM((avd=2021)::INT) OVER (PARTITION BY uarn)>=1)::INT                        AS pair_0821,
    (COUNT(DISTINCT avd) OVER (PARTITION BY uarn)=1)::INT                           AS single_period,
    LAG(scat_grp) OVER (PARTITION BY uarn ORDER BY avd)                             AS scat_grp_lag,
    LAG(scat3)     OVER (PARTITION BY uarn ORDER BY avd)                             AS scat3_lag,
    CASE WHEN LAG(scat_grp) OVER (PARTITION BY uarn ORDER BY avd) IS NULL
         THEN NULL
         ELSE (scat_grp <> LAG(scat_grp) OVER (PARTITION BY uarn ORDER BY avd))::INT
    END AS switch_it,
    CASE WHEN LAG(scat3) OVER (PARTITION BY uarn ORDER BY avd) IS NULL
         THEN NULL
         ELSE ((scat3 <> LAG(scat3) OVER (PARTITION BY uarn ORDER BY avd)) AND
               (scat_grp = LAG(scat_grp) OVER (PARTITION BY uarn ORDER BY avd)))::INT
    END AS switch_code_it
  FROM panel p
)
SELECT * FROM long
ORDER BY uarn, avd
) TO '{str(BASE/"panel_long.parquet")}' (FORMAT PARQUET);
""")

# ② 衍生子样本直接从保存的长表生成
con.execute(f"""
COPY (
  SELECT * FROM read_parquet('{str(BASE/"panel_long.parquet")}')
  WHERE balanced_3=1
) TO '{str(BASE/"panel_balanced.parquet")}' (FORMAT PARQUET);

COPY (
  SELECT * FROM read_parquet('{str(BASE/"panel_long.parquet")}')
  WHERE n_t>=2
) TO '{str(BASE/"panel_unbalanced.parquet")}' (FORMAT PARQUET);
""")


In [31]:
# 看行数
con.sql("SELECT COUNT(*) AS n FROM read_parquet('/home/jovyan/work/data/processed/voa/panel_long.parquet')").df()

# 预览前几条
con.sql("SELECT * FROM read_parquet('/home/jovyan/work/data/processed/voa/panel_long.parquet') LIMIT 5").df()


,avd,uarn,postcode,borough_i,scat,scat3,scat_grp,rv,ln_rv,n_t,balanced_3,pair_0815,pair_1521,pair_0821,single_period,scat_grp_lag,scat3_lag,switch_it,switch_code_it
0,2008,10000108000,E14 8HQ,5900,249G,249,SMALL_SHOP,1225.0,7.110696,3,1,1,1,1,0,None,None,<NA>,<NA>
1,2015,10000108000,E14 8HQ,5900,249G,249,SMALL_SHOP,1025.0,6.932448,3,1,1,1,1,0,SMALL_SHOP,249,0,0
2,2021,10000108000,E14 8HQ,5900,249G,249,SMALL_SHOP,1100.0,7.003065,3,1,1,1,1,0,SMALL_SHOP,249,0,0
3,2008,10000195000,W1J 6LY,5990,249G,249,SMALL_SHOP,40000.0,10.596635,3,1,1,1,1,0,None,None,<NA>,<NA>
4,2015,10000195000,W1J 6LY,5990,249G,249,SMALL_SHOP,95500.0,11.466882,3,1,1,1,1,0,SMALL_SHOP,249,0,0


In [8]:
import duckdb, pathlib, os

# ===== 路径 =====
ROOT    = pathlib.Path("/home/jovyan/work")
BASE    = ROOT / "data/processed/voa"
PANEL   = str(BASE / "panel_long.parquet")
ONSPD   = str(ROOT / "data/raw/ons_onspd_202405/Data/ONSPD_MAY_2024_UK.csv")

KEYMAP  = str(ROOT / "data/interim/onspd_london_keymap_pcd2.parquet")  # 缓存（pcd2）
OUT_OK  = str(BASE / "panel_long_onspd_ok.parquet")
OUT_BAL = str(BASE / "panel_balanced_onspd_ok.parquet")
OUT_UNB = str(BASE / "panel_unbalanced_onspd_ok.parquet")

# 如果能启用 spatial，则写这个 GeoParquet（双几何）；否则跳过
OUT_DUAL= str(ROOT / "data/clean/voa_panel_dualgeom.parquet")
# 如果没有 spatial，则写一个带 WKT 的备用表（非 GeoParquet，但 QGIS 可用）
OUT_WKT = str(ROOT / "data/clean/voa_panel_with_wkt.parquet")

os.makedirs(pathlib.Path(KEYMAP).parent, exist_ok=True)
os.makedirs(pathlib.Path(OUT_DUAL).parent, exist_ok=True)

con = duckdb.connect()
# 试图安装/加载 spatial；失败也继续流程
spatial_loaded = False
try:
    con.execute("INSTALL spatial; LOAD spatial;")
    spatial_loaded = True
except Exception as e:
    print("[WARN] spatial 扩展不可用，几何写出将跳过。原因：", str(e)[:120])

con.execute(f"PRAGMA threads={os.cpu_count() or 4};")
con.execute("SET memory_limit='4GB';")

# ===== 1) 仅首次：构建并缓存 伦敦 ONSPD keymap（pcd2，无空格）=====
if not os.path.exists(KEYMAP):
    con.execute(f"""
    COPY (
      WITH raw AS (
        SELECT
          UPPER(pcd2)                AS pcd2,
          lat, "long", oseast1m, osnrth1m, oslaua, doterm
        FROM read_csv_auto('{ONSPD}',
             HEADER=TRUE, SAMPLE_SIZE=-1, ALL_VARCHAR=TRUE, delim=',', quote='"')
        WHERE oslaua LIKE 'E09%'
      ),
      norm AS (
        SELECT
          regexp_replace(COALESCE(pcd2,''), '[^A-Z0-9]', '', 'g') AS key,
          TRY_CAST(lat AS DOUBLE)        AS onspd_lat,
          TRY_CAST("long" AS DOUBLE)     AS onspd_lon,
          TRY_CAST(oseast1m AS DOUBLE)   AS onspd_easting,
          TRY_CAST(osnrth1m AS DOUBLE)   AS onspd_northing,
          CASE WHEN NULLIF(doterm,'') IS NULL THEN 0 ELSE 1 END AS is_term,
          NULLIF(doterm,'') AS doterm
        FROM raw
        WHERE pcd2 IS NOT NULL AND pcd2 <> ''
      ),
      pick AS (
        SELECT * FROM (
          SELECT *,
                 ROW_NUMBER() OVER (PARTITION BY key
                   ORDER BY is_term ASC, doterm DESC NULLS LAST) AS rn
          FROM norm
        ) WHERE rn=1
      )
      SELECT key, onspd_lat, onspd_lon, onspd_easting, onspd_northing
      FROM pick
      WHERE key <> ''
    ) TO '{KEYMAP}' (FORMAT PARQUET);
    """)
    print("[BUILD] keymap (pcd2) →", KEYMAP)
else:
    print("[CACHE] using keymap (pcd2) →", KEYMAP)

# ===== 2) 与 panel 连接（pc 无空格 ↔ pcd2 无空格）→ 组内回填（2021→2015→2008）→ 剔除未匹配 =====
con.execute(f"""
CREATE TEMP VIEW pnl AS
SELECT *,
       UPPER(regexp_replace(COALESCE(postcode,''), '[^A-Z0-9]', '', 'g')) AS pc_ns
FROM read_parquet('{PANEL}');

CREATE TEMP VIEW joined AS
SELECT p.*, k.onspd_lat, k.onspd_lon, k.onspd_easting, k.onspd_northing
FROM pnl p
LEFT JOIN read_parquet('{KEYMAP}') k
ON p.pc_ns = k.key;

-- 候选坐标
CREATE TEMP VIEW coord_candidates AS
SELECT uarn, avd, onspd_lat, onspd_lon, onspd_easting, onspd_northing
FROM joined
WHERE onspd_easting IS NOT NULL AND onspd_northing IS NOT NULL;

-- 每个 uarn 选“最佳坐标”：2021 > 2015 > 2008
CREATE TEMP VIEW coord_pick AS
SELECT * FROM (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY uarn
           ORDER BY CASE avd WHEN 2021 THEN 1 WHEN 2015 THEN 2 ELSE 3 END
         ) AS rn
  FROM coord_candidates
) WHERE rn=1;

-- 回填并过滤掉仍缺失的行
CREATE TEMP VIEW ok AS
SELECT
  j.* REPLACE (
    COALESCE(j.onspd_lat,      c.onspd_lat)      AS onspd_lat,
    COALESCE(j.onspd_lon,      c.onspd_lon)      AS onspd_lon,
    COALESCE(j.onspd_easting,  c.onspd_easting)  AS onspd_easting,
    COALESCE(j.onspd_northing, c.onspd_northing) AS onspd_northing
  )
FROM joined j
LEFT JOIN coord_pick c USING (uarn)
WHERE COALESCE(j.onspd_easting, c.onspd_easting) IS NOT NULL
  AND COALESCE(j.onspd_northing, c.onspd_northing) IS NOT NULL;
""")

# 主表 + 子样本
con.execute(f"COPY (SELECT * FROM ok)                  TO '{OUT_OK}'  (FORMAT PARQUET);")
con.execute(f"COPY (SELECT * FROM ok WHERE balanced_3=1) TO '{OUT_BAL}' (FORMAT PARQUET);")
con.execute(f"COPY (SELECT * FROM ok WHERE n_t>=2)       TO '{OUT_UNB}' (FORMAT PARQUET);")



[CACHE] using keymap (pcd2) → /home/jovyan/work/data/interim/onspd_london_keymap_pcd2.parquet


In [13]:
from pathlib import Path
ROOT    = Path("/home/jovyan/work")
OUT_OK  = str(ROOT / "data/processed/voa/panel_long_onspd_ok.parquet")
OUT_DUAL= str(ROOT / "data/clean/voa_panel_dualgeom.parquet")

# 读入 ok 表
con.execute(f"CREATE OR REPLACE TEMP VIEW ok AS SELECT * FROM read_parquet('{OUT_OK}');")

# 写入：geom_27700（英网）+ geometry（WGS84）
con.execute(f"""
COPY (
  SELECT
    *,
    -- 用“27700→27700”的恒等变换给几何打上 SRID=27700
    ST_Transform(
      ST_GeomFromText('POINT(' || CAST(onspd_easting AS VARCHAR) || ' ' || CAST(onspd_northing AS VARCHAR) || ')'),
      'EPSG:27700','EPSG:27700'
    ) AS geom_27700,
    -- 从 27700 转为 4326
    ST_Transform(
      ST_GeomFromText('POINT(' || CAST(onspd_easting AS VARCHAR) || ' ' || CAST(onspd_northing AS VARCHAR) || ')'),
      'EPSG:27700','EPSG:4326'
    ) AS geometry
  FROM ok
) TO '{OUT_DUAL}' (FORMAT PARQUET);
""")
print("GeoParquet written:", OUT_DUAL)


GeoParquet written: /home/jovyan/work/data/clean/voa_panel_dualgeom.parquet


In [14]:
qa = con.execute(f"""
WITH t AS (SELECT * FROM ok)
SELECT
  COUNT(*) AS rows_kept,
  SUM(onspd_easting BETWEEN 490000 AND 570000
      AND onspd_northing BETWEEN 150000 AND 200000)::DOUBLE/COUNT(*) AS pct_in_bbox_27700,
  SUM(onspd_lat BETWEEN 50 AND 52.5 AND onspd_lon BETWEEN -0.7 AND 0.5)::DOUBLE/COUNT(*) AS pct_in_bbox_4326
FROM t;
""").fetchdf()

by_avd = con.execute("SELECT avd, COUNT(*) AS n FROM ok GROUP BY avd ORDER BY avd;").fetchdf()

print("[OK] 主表:", OUT_OK)
print("[OK] 子样本:", OUT_BAL, OUT_UNB)
print("[QA] 范围与规模：\n", qa)
print("[QA] 分期规模：\n", by_avd)

[OK] 主表: /home/jovyan/work/data/processed/voa/panel_long_onspd_ok.parquet
[OK] 子样本: /home/jovyan/work/data/processed/voa/panel_balanced_onspd_ok.parquet /home/jovyan/work/data/processed/voa/panel_unbalanced_onspd_ok.parquet
[QA] 范围与规模：
    rows_kept  pct_in_bbox_27700  pct_in_bbox_4326
0     270229           0.999819               1.0
[QA] 分期规模：
     avd      n
0  2008  90084
1  2015  90217
2  2021  89928


In [18]:
import duckdb, pathlib, os

ROOT = pathlib.Path("/home/jovyan/work")

# 输入（点表 + 区域）
P_POINTS = str(ROOT/"data/processed/voa/panel_long_onspd_ok.parquet")  # 包含 onspd_easting/onspd_northing
P_CAZ    = str(ROOT/"data/interim/gla_caz_boundary_2023_27700.gpkg")
P_TC     = str(ROOT/"data/interim/gla_town_centre_boundaries_2023_27700.gpkg")

# 输出
OUT_TAB  = str(ROOT/"data/processed/voa/panel_long_onspd_enriched_ctonly.parquet")
OUT_GEO  = str(ROOT/"data/clean/voa_panel_dualgeom_ctonly.parquet")
os.makedirs(pathlib.Path(OUT_TAB).parent, exist_ok=True)
os.makedirs(pathlib.Path(OUT_GEO).parent, exist_ok=True)

con = duckdb.connect()
con.execute("LOAD spatial;")

# 1) 生成 27700 点几何（保留所有点）
con.execute(f"""
CREATE OR REPLACE TEMP VIEW pts AS
SELECT
  *,
  ST_Transform(
    ST_GeomFromText('POINT(' || CAST(onspd_easting AS VARCHAR) || ' ' || CAST(onspd_northing AS VARCHAR) || ')'),
    'EPSG:27700','EPSG:27700'
  ) AS geom_27700
FROM read_parquet('{P_POINTS}');
""")

# 2) 读取 CAZ / Town Centre（这些 GPKG 的几何列名为 geom）
# 如需指定图层：ST_Read('{P_CAZ}', layer='真实图层名')
con.execute(f"""
CREATE OR REPLACE TEMP VIEW caz AS
SELECT ST_Transform(geom,'EPSG:27700','EPSG:27700') AS geom
FROM ST_Read('{P_CAZ}');
""")
con.execute(f"""
CREATE OR REPLACE TEMP VIEW tc AS
SELECT ST_Transform(geom,'EPSG:27700','EPSG:27700') AS geom
FROM ST_Read('{P_TC}');
""")

# 3) 空间叠加：标记 in_caz / in_town_centre（0/1）
con.execute("""
CREATE OR REPLACE TEMP VIEW pts_flagged AS
SELECT
  p.*,
  CASE WHEN EXISTS (SELECT 1 FROM caz c WHERE ST_Intersects(p.geom_27700, c.geom)) THEN 1 ELSE 0 END AS in_caz,
  CASE WHEN EXISTS (SELECT 1 FROM tc  t WHERE ST_Intersects(p.geom_27700, t.geom)) THEN 1 ELSE 0 END AS in_town_centre
FROM pts p;
""")

# 4) 写出平面表
con.execute(f"""
COPY (SELECT * FROM pts_flagged)
TO '{OUT_TAB}' (FORMAT PARQUET);
""")

# 5) 写出双几何 GeoParquet：geom_27700（分析/缓冲用）+ geometry（WGS84）
con.execute(f"""
COPY (
  SELECT
    *,
    ST_Transform(geom_27700,'EPSG:27700','EPSG:4326') AS geometry
  FROM pts_flagged
) TO '{OUT_GEO}' (FORMAT PARQUET);
""")

# 6) 简要 QA
qa = con.execute(f"""
WITH t AS (SELECT * FROM read_parquet('{OUT_TAB}'))
SELECT
  COUNT(*) AS rows,
  AVG(in_caz::DOUBLE)         AS share_in_caz,
  AVG(in_town_centre::DOUBLE) AS share_in_town_centre
FROM t;
""").fetchdf()

print(qa)
print("Wrote:\n -", OUT_TAB, "\n -", OUT_GEO)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

     rows  share_in_caz  share_in_town_centre
0  270229      0.108094              0.423389
Wrote:
 - /home/jovyan/work/data/processed/voa/panel_long_onspd_enriched_ctonly.parquet 
 - /home/jovyan/work/data/clean/voa_panel_dualgeom_ctonly.parquet


In [28]:
import duckdb, pathlib, os

ROOT = pathlib.Path("/home/jovyan/work")

# —— 输入 —— 
# 如果你刚才已跑出 CAZ/TC 版表，就用它；否则把 P_POINTS 改回 panel_long_onspd_ok.parquet 也行
P_POINTS = str(ROOT/"data/processed/voa/panel_long_onspd_enriched_ctonly.parquet")
P_PTAL   = str(ROOT/"data/interim/gla_ptal_grid_2015_27700.gpkg")  # 几何列名为 geom

# —— 输出 —— 
OUT_TAB  = str(ROOT/"data/processed/voa/panel_long_onspd_enriched_ct_ptal.parquet")
OUT_GEO  = str(ROOT/"data/clean/voa_panel_dualgeom_ct_ptal.parquet")
os.makedirs(pathlib.Path(OUT_TAB).parent, exist_ok=True)
os.makedirs(pathlib.Path(OUT_GEO).parent, exist_ok=True)

con = duckdb.connect()
con.execute("LOAD spatial;")

# 1) 点：构造 27700 几何（保留所有点）
con.execute(f"""
CREATE OR REPLACE TEMP VIEW pts AS
SELECT *
FROM read_parquet('{P_POINTS}');
""")

# 2) 读取 PTAL 栅格点（若报图层名问题：在 ST_Read 加 layer='你的图层名'）
con.execute(f"""
CREATE OR REPLACE TEMP VIEW ptal_raw AS
SELECT
  TRY_CAST(ai_2015 AS DOUBLE)       AS ai_2015,
  CAST(ptal_2015_band AS VARCHAR)   AS ptal_2015_band,
  TRY_CAST(ptal_2015_num AS DOUBLE) AS ptal_2015_num,
  ST_Transform(geom,'EPSG:27700','EPSG:27700') AS geom
FROM ST_Read('{P_PTAL}');
""")

# 3) 轻量近邻：按 300m 分桶，只在同桶±1 的 9 个桶里找最近
con.execute("""
CREATE OR REPLACE TEMP VIEW ptal AS
SELECT
  ai_2015, ptal_2015_band, ptal_2015_num, geom,
  CAST(FLOOR(ST_X(geom)/300.0) AS BIGINT) AS bx,
  CAST(FLOOR(ST_Y(geom)/300.0) AS BIGINT) AS by
FROM ptal_raw;

CREATE OR REPLACE TEMP VIEW pts_bucket AS
SELECT *,
       CAST(FLOOR(onspd_easting/300.0)  AS BIGINT) AS bx,
       CAST(FLOOR(onspd_northing/300.0) AS BIGINT) AS by
FROM pts;

CREATE OR REPLACE TEMP VIEW cand AS
SELECT
  p.uarn, p.avd, p.geom_27700,
  g.ai_2015, g.ptal_2015_num, g.ptal_2015_band, g.geom AS ptal_geom,
  ST_Distance(p.geom_27700, g.geom) AS d
FROM pts_bucket p
JOIN ptal g
  ON ABS(g.bx - p.bx) <= 1 AND ABS(g.by - p.by) <= 1;

CREATE OR REPLACE TEMP VIEW nearest_ptal AS
SELECT *
FROM (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY uarn, avd ORDER BY d ASC) AS rn
  FROM cand
) WHERE rn=1;
""")

# 4) 合并并写出（平面表）
con.execute(f"""
COPY (
  SELECT
    p.*,
    n.ptal_2015_num,
    n.ptal_2015_band,
    n.ai_2015,
    n.d AS ptal_dist_m
  FROM pts p
  LEFT JOIN nearest_ptal n USING (uarn, avd)
) TO '{OUT_TAB}' (FORMAT PARQUET);
""")

# 5) 写 GeoParquet（双几何：geom_27700 + geometry/WGS84）
con.execute(f"""
COPY (
  SELECT
    *,
    ST_Transform(geom_27700,'EPSG:27700','EPSG:4326') AS geometry
  FROM read_parquet('{OUT_TAB}')
) TO '{OUT_GEO}' (FORMAT PARQUET);
""")

# 6) 简要 QA（覆盖率 & 最近邻距离）
qa = con.execute(f"""
WITH t AS (SELECT * FROM read_parquet('{OUT_TAB}'))
SELECT
  COUNT(*) AS rows,
  SUM(CASE WHEN ptal_2015_num IS NOT NULL THEN 1 ELSE 0 END)::DOUBLE/COUNT(*) AS ptal_coverage,
  -- 只在非空上算分位数
  median(ptal_dist_m)                        AS med_ptal_dist_m,
  quantile(ptal_dist_m, 0.9)                 AS p90_ptal_dist_m
FROM t
WHERE ptal_dist_m IS NOT NULL;
""").fetchdf()
print(qa)



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

     rows  ptal_coverage  med_ptal_dist_m  p90_ptal_dist_m
0  270229            1.0        39.446166        55.803226


In [29]:
import duckdb, pandas as pd, pathlib
ROOT = pathlib.Path("/home/jovyan/work")
TAB  = str(ROOT/"data/processed/voa/panel_long_onspd_enriched_ct_ptal.parquet")
con  = duckdb.connect()

# 取消列折叠
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 180)

# ① 全列预览（按 uarn、avd 排序，前 12 行）
con.execute(f"""
SELECT *
FROM read_parquet('{TAB}')
ORDER BY uarn, avd
LIMIT 12
""").fetchdf()


,avd,uarn,postcode,borough_i,scat,scat3,scat_grp,rv,ln_rv,n_t,balanced_3,pair_0815,pair_1521,pair_0821,single_period,scat_grp_lag,scat3_lag,switch_it,switch_code_it,pc_ns,onspd_lat,onspd_lon,onspd_easting,onspd_northing,geom_27700,in_caz,in_town_centre,ptal_2015_num,ptal_2015_band,ai_2015,ptal_dist_m
0,2008,10000108000,E14 8HQ,5900,249G,249,SMALL_SHOP,1225.0,7.110696,3,1,1,1,1,0,None,None,<NA>,<NA>,E148HQ,51.511253,-0.028747,536891.0,180989.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 118, 98, 32, 65, 0...",0,0,9.0,6a,25.221447,59.033889
1,2015,10000108000,E14 8HQ,5900,249G,249,SMALL_SHOP,1025.0,6.932448,3,1,1,1,1,0,SMALL_SHOP,249,0,0,E148HQ,51.511253,-0.028747,536891.0,180989.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 118, 98, 32, 65, 0...",0,0,9.0,6a,25.221447,59.033889
2,2021,10000108000,E14 8HQ,5900,249G,249,SMALL_SHOP,1100.0,7.003065,3,1,1,1,1,0,SMALL_SHOP,249,0,0,E148HQ,51.511253,-0.028747,536891.0,180989.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 118, 98, 32, 65, 0...",0,0,9.0,6a,25.221447,59.033889
3,2008,10000195000,W1J 6LY,5990,249G,249,SMALL_SHOP,40000.0,10.596635,3,1,1,1,1,0,None,None,<NA>,<NA>,W1J6LY,51.511264,-0.144687,528846.0,180780.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 156, 35, 32, 65, 0...",1,0,10.0,6b,53.260808,28.017851
4,2015,10000195000,W1J 6LY,5990,249G,249,SMALL_SHOP,95500.0,11.466882,3,1,1,1,1,0,SMALL_SHOP,249,0,0,W1J6LY,51.511264,-0.144687,528846.0,180780.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 156, 35, 32, 65, 0...",1,0,10.0,6b,53.260808,28.017851
5,2021,10000195000,W1J 6LY,5990,249G,249,SMALL_SHOP,76000.0,11.238489,3,1,1,1,1,0,SMALL_SHOP,249,0,0,W1J6LY,51.511264,-0.144687,528846.0,180780.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 156, 35, 32, 65, 0...",1,0,10.0,6b,53.260808,28.017851
6,2008,10000201000,SW1E 6SQ,5990,249G,249,SMALL_SHOP,520000.0,13.161584,2,0,1,0,0,0,None,None,<NA>,<NA>,SW1E6SQ,51.497326,-0.138467,529317.0,179241.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 74, 39, 32, 65, 0,...",1,0,10.0,6b,77.749249,30.083218
7,2015,10000201000,SW1E 6SQ,5990,249G,249,SMALL_SHOP,530000.0,13.180632,2,0,1,0,0,0,SMALL_SHOP,249,0,0,SW1E6SQ,51.497326,-0.138467,529317.0,179241.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 74, 39, 32, 65, 0,...",1,0,10.0,6b,77.749249,30.083218
8,2008,10000223000,SW1P 1QN,5990,249G,249,SMALL_SHOP,133000.0,11.798104,3,1,1,1,1,0,None,None,<NA>,<NA>,SW1P1QN,51.496430,-0.137412,529277.0,178945.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 250, 38, 32, 65, 0...",1,0,10.0,6b,74.775150,32.756679
9,2015,10000223000,SW1P 1QN,5990,249G,249,SMALL_SHOP,154000.0,11.944708,3,1,1,1,1,0,SMALL_SHOP,249,0,0,SW1P1QN,51.496430,-0.137412,529277.0,178945.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 250, 38, 32, 65, 0...",1,0,10.0,6b,74.775150,32.756679


In [50]:
import duckdb, pandas as pd, pathlib

ROOT = pathlib.Path("/home/jovyan/work")
LONG_IN   = str(ROOT/"data/processed/voa/panel_long_onspd_enriched_ct_ptal.parquet")
ONSPD_CSV = str(ROOT/"data/raw/ons_onspd_202405/Data/ONSPD_MAY_2024_UK.csv")
IMD_XLSX  = str(ROOT/"data/raw/dluhc_imd_2019_tables.xlsx")

# 单一连接
con = duckdb.connect()

# 1) 基表 + 规范邮编（同一个 con）
con.execute(f"""
CREATE OR REPLACE TEMP VIEW voa AS
SELECT *,
       UPPER(regexp_replace(COALESCE(postcode,''),'[^A-Z0-9]','','g')) AS pc_nospace
FROM read_parquet('{LONG_IN}');
""")

# 2) 构建并注册 onspd_map（pc_nospace -> lsoa11cd）
# 先拿到需要的邮编键，减少 ONSPD 体量
pc_need = con.execute("SELECT DISTINCT pc_nospace FROM voa WHERE pc_nospace<>''").fetchdf()['pc_nospace'].tolist()
pc_need = set(pc_need)

want = {"pcd","pcd2","pcds","lsoa11","lsoa11cd","lsoa21","lsoa21cd","doterm"}
onspd = pd.read_csv(ONSPD_CSV, dtype=str, low_memory=False, usecols=lambda c: c.lower() in want)

for c in ("pcd","pcd2","pcds"):
    if c in onspd.columns:
        onspd[c] = onspd[c].astype(str).str.replace(r"[^A-Za-z0-9]","", regex=True).str.upper()

# 选择 LSOA 列（优先 2011）
lsoa_col = next((c for c in ("lsoa11cd","lsoa11","lsoa21cd","lsoa21") if c in onspd.columns), None)
assert lsoa_col is not None, "ONSPD 中未找到 LSOA 列"

frames = []
for c in [x for x in ("pcd","pcd2","pcds") if x in onspd.columns]:
    tmp = onspd[[c, lsoa_col, "doterm"]].rename(columns={c:"key", lsoa_col:"lsoa_code"})
    frames.append(tmp)
keymap = pd.concat(frames, ignore_index=True)
keymap["key"] = keymap["key"].fillna("")
keymap = keymap[keymap["key"].isin(pc_need)]            # 只保留需要的邮编
keymap["is_current"] = keymap["doterm"].isna()
keymap = keymap.sort_values(["key","is_current","doterm"], ascending=[True, False, False]).drop_duplicates("key", keep="first")
if lsoa_col.startswith("lsoa11"):
    keymap = keymap[keymap["lsoa_code"].astype(str).str.match(r"^E010\d{5}$", na=False)]
keymap = keymap.rename(columns={"key":"pc_nospace", "lsoa_code":"lsoa11cd"})[["pc_nospace","lsoa11cd"]]

con.register("onspd_map", keymap)

# 3) 构建并注册 imd2019（LSOA11 -> rank/decile）
imd_raw = pd.read_excel(IMD_XLSX, sheet_name="IMD2019", dtype=str)
cols = {c.lower(): c for c in imd_raw.columns}
def find_col(keys):  # keys 是若干关键词
    for k,c in cols.items():
        if all(kw in k for kw in keys):
            return c
    raise KeyError(keys)

lsoa_c = cols.get("lsoa code (2011)") or find_col(["lsoa","2011"])
rank_c = cols.get("index of multiple deprivation (imd) rank")   or find_col(["imd","rank"])
deci_c = cols.get("index of multiple deprivation (imd) decile") or find_col(["imd","decile"])

imd = (imd_raw[[lsoa_c, rank_c, deci_c]]
       .rename(columns={lsoa_c:"lsoa11cd", rank_c:"imd2019_rank", deci_c:"imd2019_decile"}))
imd["lsoa11cd"]       = imd["lsoa11cd"].astype(str).str.strip()
imd["imd2019_rank"]   = pd.to_numeric(imd["imd2019_rank"], errors="coerce").astype("Int64")
imd["imd2019_decile"] = pd.to_numeric(imd["imd2019_decile"], errors="coerce").astype("Int64")

con.register("imd2019", imd)

# 4) 合并（同一个 con，不要再 duckdb.connect()）
con.execute("""
CREATE OR REPLACE TEMP VIEW voa_lsoa AS
SELECT v.*, m.lsoa11cd
FROM voa v
LEFT JOIN onspd_map m USING (pc_nospace);
""")

con.execute("""
CREATE OR REPLACE TEMP VIEW voa_imd AS
SELECT v.*, i.imd2019_rank, i.imd2019_decile
FROM voa_lsoa v
LEFT JOIN imd2019 i USING (lsoa11cd);
""")

# 5) QA + 样例
print(con.execute("""
WITH t AS (SELECT * FROM voa_imd)
SELECT COUNT(*) AS rows,
       SUM(lsoa11cd IS NULL)::DOUBLE/COUNT(*)       AS miss_lsoa_rate,
       SUM(imd2019_decile IS NULL)::DOUBLE/COUNT(*) AS miss_imd_rate
FROM t;
""").fetchdf())

print(con.execute("""
SELECT avd, uarn, postcode, lsoa11cd, imd2019_decile
FROM voa_imd
WHERE lsoa11cd IS NOT NULL
LIMIT 10
""").fetchdf())

import os, pathlib
ROOT = pathlib.Path("/home/jovyan/work")

OUT_LONG = str(ROOT/"data/processed/voa/panel_long_onspd_enriched_ct_ptal_imd.parquet")
OUT_BAL  = str(ROOT/"data/processed/voa/panel_balanced_onspd_ct_ptal_imd.parquet")
OUT_UNB  = str(ROOT/"data/processed/voa/panel_unbalanced_onspd_ct_ptal_imd.parquet")
OUT_CSV  = str(ROOT/"data/clean/voa_panel_enriched_ct_ptal_imd.csv")

for p in (OUT_LONG, OUT_BAL, OUT_UNB, OUT_CSV):
    os.makedirs(pathlib.Path(p).parent, exist_ok=True)

# 写出：完整长表 + CSV
con.execute(f"COPY (SELECT * FROM voa_imd) TO '{OUT_LONG}' (FORMAT PARQUET);")
con.execute(f"COPY (SELECT * FROM voa_imd) TO '{OUT_CSV}'  (HEADER, DELIM ',');")

# 写出：子样本
con.execute(f"COPY (SELECT * FROM voa_imd WHERE balanced_3=1) TO '{OUT_BAL}' (FORMAT PARQUET);")
con.execute(f"COPY (SELECT * FROM voa_imd WHERE n_t>=2)   TO '{OUT_UNB}' (FORMAT PARQUET);")

print("[OK] wrote:")
print(" -", OUT_LONG)
print(" -", OUT_BAL)
print(" -", OUT_UNB)
print(" -", OUT_CSV)



     rows  miss_lsoa_rate  miss_imd_rate
0  270229             0.0            0.0
    avd        uarn  postcode   lsoa11cd  imd2019_decile
0  2021   826575000    N1 7PT  E01001783               2
1  2021   826600000  SW11 3BU  E01004579               5
2  2021  8267187000   BR7 5AQ  E01000692               9
3  2021    82788068    E5 0PD  E01001734               2
4  2021  8282730000   SW9 9TQ  E01003053               4
5  2021    83022051   RM8 1DJ  E01000109               4
6  2021  8306951000   HA0 3NJ  E01000564               6
7  2021  8307372000  WC2E 8RA  E01004734               3
8  2021  8321076000    E1 2DA  E01004321               2
9  2021    83347065   UB4 9AX  E01002542               3
[OK] wrote:
 - /home/jovyan/work/data/processed/voa/panel_long_onspd_enriched_ct_ptal_imd.parquet
 - /home/jovyan/work/data/processed/voa/panel_balanced_onspd_ct_ptal_imd.parquet
 - /home/jovyan/work/data/processed/voa/panel_unbalanced_onspd_ct_ptal_imd.parquet
 - /home/jovyan/work/data/cle

In [52]:
import duckdb, pandas as pd, pathlib
ROOT = pathlib.Path("/home/jovyan/work")
TAB  = str(ROOT/"data/processed/voa/panel_long_onspd_enriched_ct_ptal_imd.parquet")
con  = duckdb.connect()

# 取消列折叠
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 180)

# ① 全列预览（按 uarn、avd 排序，前 12 行）
con.execute(f"""
SELECT *
FROM read_parquet('{TAB}')
ORDER BY uarn, avd
LIMIT 5
""").fetchdf()

,avd,uarn,postcode,borough_i,scat,scat3,scat_grp,rv,ln_rv,n_t,balanced_3,pair_0815,pair_1521,pair_0821,single_period,scat_grp_lag,scat3_lag,switch_it,switch_code_it,pc_ns,onspd_lat,onspd_lon,onspd_easting,onspd_northing,geom_27700,in_caz,in_town_centre,ptal_2015_num,ptal_2015_band,ai_2015,ptal_dist_m,pc_nospace,lsoa11cd,imd2019_rank,imd2019_decile
0,2008,10000108000,E14 8HQ,5900,249G,249,SMALL_SHOP,1225.0,7.110696,3,1,1,1,1,0,None,None,<NA>,<NA>,E148HQ,51.511253,-0.028747,536891.0,180989.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 118, 98, 32, 65, 0...",0,0,9.0,6a,25.221447,59.033889,E148HQ,E01004255,5983,2
1,2015,10000108000,E14 8HQ,5900,249G,249,SMALL_SHOP,1025.0,6.932448,3,1,1,1,1,0,SMALL_SHOP,249,0,0,E148HQ,51.511253,-0.028747,536891.0,180989.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 118, 98, 32, 65, 0...",0,0,9.0,6a,25.221447,59.033889,E148HQ,E01004255,5983,2
2,2021,10000108000,E14 8HQ,5900,249G,249,SMALL_SHOP,1100.0,7.003065,3,1,1,1,1,0,SMALL_SHOP,249,0,0,E148HQ,51.511253,-0.028747,536891.0,180989.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 118, 98, 32, 65, 0...",0,0,9.0,6a,25.221447,59.033889,E148HQ,E01004255,5983,2
3,2008,10000195000,W1J 6LY,5990,249G,249,SMALL_SHOP,40000.0,10.596635,3,1,1,1,1,0,None,None,<NA>,<NA>,W1J6LY,51.511264,-0.144687,528846.0,180780.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 156, 35, 32, 65, 0...",1,0,10.0,6b,53.260808,28.017851,W1J6LY,E01033595,16446,6
4,2015,10000195000,W1J 6LY,5990,249G,249,SMALL_SHOP,95500.0,11.466882,3,1,1,1,1,0,SMALL_SHOP,249,0,0,W1J6LY,51.511264,-0.144687,528846.0,180780.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 156, 35, 32, 65, 0...",1,0,10.0,6b,53.260808,28.017851,W1J6LY,E01033595,16446,6
